In [23]:
import pandas as pd
import numpy as np
import rdflib
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline

## Loading knowledge graph

In [6]:
graph = rdflib.Graph()
graph.parse("data/reference-kg.nt")

training_data = rdflib.Graph()
training_data.parse("data/fokg-sw-train-2024.nt")

<Graph identifier=N0ceff0efcfd04ae99510c3a3e4c96cee (<class 'rdflib.graph.Graph'>)>

## Converting to pykeen format

In [ ]:
data = np.array([(s, p, o) for s, p, o in graph])
pykeen_data = TriplesFactory.from_labeled_triples(data)


## Learning the Embeddings

In [24]:
training, testing, validation = pykeen_data.split([0.8, 0.1, 0.1])
result = pipeline(
    training=training,
    testing=testing,
    validation=validation,
    model='TransE',
)


using automatically assigned random_state=680947110
No random seed is specified. Setting to 740805541.
No cuda devices were available. The model runs on CPU
Training epochs on cpu: 100%|██████████| 5/5 [03:31<00:00, 42.38s/epoch, loss=0.0748, prev_loss=0.0999]
Evaluating on cpu:   0%|          | 0.00/67.6k [00:00<?, ?triple/s]WARNING:torch_max_mem.api:Encountered tensors on device_types={'cpu'} while only ['cuda'] are considered safe for automatic memory utilization maximization. This may lead to undocumented crashes (but can be safe, too).
Evaluating on cpu:   9%|▉         | 6.24k/67.6k [02:33<25:05, 40.7triple/s]


KeyboardInterrupt: 